## Modelo de scattering con interfaces rugosas basado en 


- Modeling of EM Wave Coherent Scattering From a Rough Multilayered Medium With the Scalar Kirchhoff Approximation for GPR Applications,

    Nicolas Pinel, Cédric Le Bastard, Christophe Bourlier, IEEE Transactions on Geoscience and Remote Sensing, 58(3), 1654-1664.



- DEGREE OF ROUGHNESS OF ROUGH LAYERS: EXTENSIONS OF THE RAYLEIGH ROUGHNESS CRITERION AND SOME APPLICATIONS,

    N. Pinel, C. Bourlier, and J. Saillard, Progress In Electromagnetics Research B, Vol. 19, 41–63, 2010



In [2]:
import numpy as np

## Incidencia normal

In [ ]:
def R_pinel_2L(ep1,ep2,d1,s0,s1,f):
        
                
    r01 = (1-np.sqrt(ep1))/(1+np.sqrt(ep1))
    t01 = 2/(1+np.sqrt(ep1))
    t10 = 2*np.sqrt(ep1)/(1+np.sqrt(ep1))
    r12 =  (np.sqrt(ep1)-np.sqrt(ep2))/(np.sqrt(ep1)+np.sqrt(ep2))

    k0z = 2*np.pi*f/3e8
    k1z = np.sqrt(np.real(ep1))*k0z

    Es = r01*np.exp(-2*(k0z*s1)**2) + t01*r12*t10*np.exp(-2j*k1z*d1)*np.exp(-(k0z-k1z)**2*s0**2)*np.exp(-2*(k1z*s1)**2)
    
    return Es
        
    
def R_pinel_3L(ep1,ep2,ep3,d1,d2,s0,s1,s2,f):
    
            
    r01 = (1-np.sqrt(ep1))/(1+np.sqrt(ep1))
    t01 = 2/(1+np.sqrt(ep1))
    t10 = 2*np.sqrt(ep1)/(1+np.sqrt(ep1))

    r12 =  (np.sqrt(ep1)-np.sqrt(ep2))/(np.sqrt(ep1)+np.sqrt(ep2))
    t12 = 2*np.sqrt(ep1)/(np.sqrt(ep1)+np.sqrt(ep2))
    t21 = 2*np.sqrt(ep2)/(np.sqrt(ep1)+np.sqrt(ep2))

    r23 = (np.sqrt(ep2)-np.sqrt(ep3))/(np.sqrt(ep2)+np.sqrt(ep3))

    k0z = 2*np.pi*f/3e8
    k1z = np.sqrt(np.real(ep1))*k0z
    k2z = np.sqrt(np.real(ep2))*k0z

    Es = r01*np.exp(-2*(k0z*s0)**2) + t01*r12*t10*np.exp(-2j*k1z*d1)*np.exp(-(k0z-k1z)**2*s0**2)*np.exp(-2*(k1z*s1)**2)+\
        t01*t12*r23*t21*t10*np.exp(-2j*k1z*d1)*np.exp(-2j*k2z*(d2-d1))*np.exp(-(s0*(k0z-k1z))**2)*np.exp(-(s1*(k1z-k2z))**2)*np.exp(-2*(s2*k2z)**2)

    return Es

## $\theta_i \neq 0$

### Una capa
Seguimos el modelo del paper \"Degree of roughness of rough layers\" - pinel (2010)

\begin{equation}
<E_t>=<E_{r1}>+<E_{r2}>
\end{equation}

\begin{cases}
<E_{r1}>=R_{01}E_ie^{-2k_{0z}^2\\sigma_a^2}\\
<E_{r2}>=T_{01}R_{12}T_{10}E_ie^{i2k_{1z}d}e^{-(k_{0z}-k_{1z})^2\\sigma_a^2}e^{-k_{1z}^2\\sigma_b^2}
\end{cases}

donde

$d$ es el ancho de la capa

$R_{ij}$ y $T_{ij}$ son los coeficienes de reflexion de fresnel

$k_{0z}=\frac{2\\pi f}{c}\sqrt{\epsilon_0}cos(\theta_0)$ (obs: aire $\epsilon_0=1$)

$k_{1z}=\frac{2\pi f}{c}\sqrt{\epsilon_1}cos(\theta_1)$ con $cos(\theta_1)=\sqrt{1-\frac{\epsilon_0}{\epsilon_1}sen^2(\theta_0)}$

$\sigma_a$ y $\sigma_b$ son la altura RMS (root mean square) de las capas

In [ ]:
def Rf_2(ep1,ep2,d,tita,f,rms1,rms2):
    #---------coef fresnel-----------
    #por snell
    tita_1 = np.arcsin(np.sin(tita)/np.sqrt(ep1))
    
    R01 = R_ij(1,ep1,tita, tita_1)
    T01 = T_ij(1,ep1,tita,tita_1)
    T10 = T_ij(ep1,1,tita_1,tita)
    
    #por snell
    tita_2 = np.arcsin(np.sqrt(ep1/ep2)*np.sin(tita_1))
    R12 = R_ij(ep1,ep2,tita_1,tita_2)
    #--------------------------------
    
    #--------kiz---------------------
    k0z = (2*np.pi*f*np.cos(tita))/3e8
    k1z = (2*np.pi*f+np.sqrt(np.real(ep1))*np.cos(tita_1))/3e8
    #--------------------------------
    
    Rf1 = R01*np.exp(-2*k0z**2*rms1**2)
    Rf2 = T01*R12*T10*np.exp(2*1j*k1z*d)*np.exp(-(k0z-k1z)**2*rms1**2)*np.exp(-2*k1z**2*rms2**2)
    Rft = Rf1+Rf2
    return Rft

### Una capa con 3 rebotes

\begin{equation}
<E_t>=<E_{r1}>+<E_{r2}>+<E_{r3}>
\end{equation}

\begin{cases}
<E_{r1}>=R_{01}E_ie^{-2k_{0z}^2\sigma_a^2}\\
<E_{r2}>=T_{01}R_{12}T_{10}E_ie^{i2k_{1z}d}e^{-(k_{0z}-k_{1z})^2\sigma_a^2}e^{-k_{1z}^2\sigma_b^2}\\
<E_{r3}>=T_{01}R_{12}R_{10}R_{12}T_{10}E_ie^{i4k_{1z}d}e^{-(k_{0z}-k_{1z})^2\sigma_a^2}e^{-2k_{1z}^2(2\sigma_b^2+\sigma_a^2)}
\end{cases}

donde

$d$ es el ancho de la capa

$R_{ij}$ y $T_{ij}$ son los coeficienes de reflexion de fresnel

$k_{0z}=\frac{2\pi f}{c}\sqrt{\epsilon_0}cos(\theta_0)$ (obs: aire $\epsilon_0=1$)

$k_{1z}=\frac{2\pi f}{c}\sqrt{\epsilon_1}cos(\theta_1)$ con $cos(\theta_1)=\sqrt{1-\frac{\epsilon_0}{\epsilon_1}sen^2(\theta_0)}$

$\sigma_a$ y $\sigma_b$ son la altura RMS (root mean square) de las capas


In [1]:
def Rf_2b(ep1,ep2,d,tita,f,rms1,rms2):
    #---------coef fresnel-----------
    #por snell
    tita_1 = np.arcsin(np.sin(tita)/np.sqrt(ep1))
    
    R01 = R_ij(1,ep1,tita, tita_1)
    R10 = R_ij(ep1,1,tita_1,tita)
    T01 = T_ij(1,ep1,tita,tita_1)
    T10 = T_ij(ep1,1,tita_1,tita)
    
    #por snell
    tita_2 = np.arcsin(np.sqrt(ep1/ep2)*np.sin(tita_1))
    R12 = R_ij(ep1,ep2,tita_1,tita_2)
    #--------------------------------
    
    #--------kiz---------------------
    k0z = (2*np.pi*f*np.cos(tita))/3e8
    k1z = (2*np.pi*f+np.sqrt(np.real(ep1))*np.cos(tita_1))/3e8
    #--------------------------------
    
    Rf1 = R01*np.exp(-2*k0z**2*rms1**2)
    Rf2 = T01*R12*T10*np.exp(2*1j*k1z*d)*np.exp(-(k0z-k1z)**2*rms1**2)*np.exp(-2*k1z**2*rms2**2)
    Rf3 = T01*R12**2*R10*T10*np.exp(4*1j*k1z*d)*np.exp(-(k0z-k1z)**2*rms1**2)*np.exp(-2*k1z**2*(2*rms2**2+rms1**2))
    Rft = Rf1+Rf2+Rf3
    return Rft

### Dos capas    
Seguimos el modelo del paper \Degree of roughness of rough layers\ - pinel (2010)

\begin{equation}
<E_t>=<E_{r1}>+<E_{r2}>+<E_{r3}>
\end{equation}

\begin{cases}
<E_{r1}>=R_{01} E_i e^{-2k_{0z}^2\sigma_a^2}\\
<E_{r2}>=T_{01}R_{12}T_{10} E_i e^{i2k_{1z}d_1} e^{-(k_{0z}-k_{1z})^2\sigma_a^2} e^{-k_{1z}^2\sigma_b^2}\\
<E_{r3}>=T_{01}T_{12}R_{23}T_{21}T_{10} E_i e^{i2[(k_{1z}-k_{2z})d_1+k_{2z}d_2]} e^{-(k_{0z}-k_{1z})^2\sigma_a^2} e^{-(k_{1z}-k_{2z})^2\sigma_b^2}e^{-k_{2z}^2\sigma_c^2}
\end{cases}

donde

$d1$, $d2$ son los anchos de las capas

$R_{ij}$ y $T_{ij}$ son los coeficienes de reflexion de fresnel

$k_{0z}=\frac{2\pi f}{c}\sqrt{\epsilon_0}cos(\theta_0)$ (obs: aire $\epsilon_0=1$)

$k_{1z}=\frac{2\pi f}{c}\sqrt{\epsilon_1}cos(\theta_1)$ con $cos(\theta_1)=\sqrt{1-\frac{\epsilon_0}{\epsilon_1}sen^2(\theta_0)}$

$k_{2z}=\frac{2\pi f}{c}\sqrt{\epsilon_2}cos(\theta_2)$ con $cos(\theta_2)=\sqrt{1-\frac{\epsilon_1}{\epsilon_2}sen^2(\theta_1)}$

$\sigma_a$, $\sigma_b$ y $\sigma_c$ son la altura RMS (root mean square) de las capas


In [ ]:
def Rf_3(ep1,ep2,ep3,d1,d2,tita_0,f,rms1,rms2,rms3):
    #---------coef fresnel-----------
    #por snell
    tita_1 = np.arcsin(np.sin(tita_0)/np.sqrt(ep1))
    
    R01 = R_ij(1,ep1,tita_0, tita_1)
    T01 = T_ij(1,ep1,tita_0,tita_1)
    T10 = T_ij(ep1,1,tita_1,tita_0)
    
    #por snell
    tita_2 = np.arcsin(np.sqrt(ep1/ep2)*np.sin(tita_1))
    R12 = R_ij(ep1,ep2,tita_1,tita_2)
    T12 = R_ij(ep1,ep2,tita_1,tita_2)
    T21 = R_ij(ep1,ep2,tita_1,tita_2)
    
    #por snell
    tita_3 = np.arcsin(np.sqrt(ep2/ep3)*np.sin(tita_2))
    R23 = R_ij(ep2,ep3,tita_2,tita_3)
    #--------------------------------
    
    #--------kiz---------------------
    k0z = (2*np.pi*f*np.cos(tita_0))/3e8
    k1z = (2*np.pi*f+np.sqrt(np.real(ep1))*np.cos(tita_1))/3e8
    k2z = (2*np.pi*f+np.sqrt(np.real(ep2))*np.cos(tita_2))/3e8
    #--------------------------------
    
    Rf1 = R01*np.exp(-2*k0z**2*rms1**2)
    Rf2 = T01*R12*T10*np.exp(2*1j*k1z*d1)*np.exp(-(k0z-k1z)**2*rms1**2)*np.exp(-2*k1z**2*rms2**2)
    
    phi = -[(k0z-k1z)*rms1]**2-[(k1z-k2z)*rms2]**2-2*[k2z*rms3]**2
    Rf3 = T01*T12*R23*T21*T10*np.exp(2*1j*[(k1z-k2z)*d1+k2z*d2])*np.exp(phi)
    
    Rft = Rf1+Rf2+Rf3
    return Rft